<a href="https://colab.research.google.com/github/Irenekayla/ML_Notebooks/blob/main/Cnn_AirQo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow keras
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/AirQo_Kenya/Train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/AirQo_Kenya/Test.csv')

In [ ]:

X_train['lat_long'] = X_train['site_latitude'] + X_train['site_longitude']
X_test['lat_long'] = X_test['site_latitude'] + X_test['site_longitude']


numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist() + ['lat_long']

numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, X_train.select_dtypes(include=['object']).columns)
])

preprocessor.fit(X_train)


ColumnTransformer(transformers=[('num', SimpleImputer(),
                                 ['site_latitude', 'site_longitude', 'hour',
                                  'sulphurdioxide_so2_column_number_density',
                                  'sulphurdioxide_so2_column_number_density_amf',
                                  'sulphurdioxide_so2_slant_column_number_density',
                                  'sulphurdioxide_cloud_fraction',
                                  'sulphurdioxide_sensor_azimuth_angle',
                                  'sulphurdioxide_sensor_zenith_angle',
                                  'sulphur...
                                  'nitrogendioxide_tropopause_pressure',
                                  'nitrogendioxide_absorbing_aerosol_index',
                                  'nitrogendioxide_cloud_fraction',
                                  'nitrogendioxide_sensor_altitude',
                                  'nitrogendioxide_sensor_azimuth_angle', ...]),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['\nid', 'site_id', 'city', 'country', 'date'], dtype='object'))])

In [ ]:
X_train, y_train = train_data.drop('pm2_5', axis=1), train_data['pm2_5']
X_test, y_test = test_data.drop('pm2_5', axis=1), test_data['pm2_5']

problematic_column = '\\nid'


if problematic_column in X_test.columns:
    X_test = X_test.drop(problematic_column, axis=1)


In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

In [ ]:
from sklearn.model_selection import GridSearchCV

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])


param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.05, 0.1]
}


grid_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)


grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [ ]:
print(X_test.columns)

X_test.columns = X_test.columns.str.strip()



In [ ]:

problematic_column = '\nid'


if problematic_column in X_test.columns:
    X_test = X_test.drop(problematic_column, axis=1)


y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared Score: {r2:.2f}')

KeyError: "['lat_long'] not in index"

In [ ]:
print(X_train.columns)
print(X_test.columns)
